In [ ]:
import sys

sys.path.append("../")
from helper.utils import *
from helper.cross_validation import *
from config import *
from helper.categorical_features import * 
from sklearn.cluster import KMeans
import geopandas as gpd
# %load_ext lab_black
from geopy.distance import distance
from scipy.spatial.distance import cdist
distance_m=lambda x,y : distance(x,y).m

In [ ]:
ward_ALL=gpd.read_file(join(raw_data_path,"zaf_adm_2016SADB_OCHA_SHP","zaf_admbndl_ALL_2016SADB_OCHA.shp"))
ward_ALL.head()

In [ ]:
ward_1=gpd.read_file(join(raw_data_path,"zaf_adm_2016SADB_OCHA_SHP","zaf_admbnda_adm1_2016SADB_OCHA.shp"))
ward_1.head()

In [ ]:
ward_2=gpd.read_file(join(raw_data_path,"zaf_adm_2016SADB_OCHA_SHP","zaf_admbnda_adm2_2016SADB_OCHA.shp"))
ward_2.head()

In [ ]:
ward_3=gpd.read_file(join(raw_data_path,"zaf_adm_2016SADB_OCHA_SHP","zaf_admbnda_adm3_2016SADB_OCHA.shp"))
ward_3.head()

In [ ]:
ward_4=gpd.read_file(join(raw_data_path,"zaf_adm_2016SADB_OCHA_SHP","zaf_admbnda_adm4_2016SADB_OCHA.shp"))
ward_4.head()

In [ ]:
from pyproj import Proj
pa = Proj("+proj=aea +lat_1=37.0 +lat_2=41.0 +lat_0=39.0 +lon_0=-106.55")
ward_4.crs=pa

In [ ]:
ward_4.head()

In [ ]:
ward_4["area_4"]=ward_4.geometry.area
ward_3["area_3"]=ward_3.geometry.area
ward_2["area_2"]=ward_2.geometry.area
ward_1["area_1"]=ward_1.geometry.area

In [ ]:
# clsutering ADM code  with feature slike the NL ,  housleholders ....

In [ ]:
ward_area=ward_4[["ADM4_PCODE","area_4","ADM3_PCODE"]]
ward_area=ward_area.merge(ward_3[["ADM3_PCODE","ADM2_PCODE","area_3"]],how="left",on=["ADM3_PCODE"])
ward_area=ward_area.merge(ward_2[["ADM2_PCODE","ADM1_PCODE","area_2"]],how="left",on=["ADM2_PCODE"])
ward_area=ward_area.merge(ward_1[["ADM1_PCODE","area_1"]],how="left",on=["ADM1_PCODE"])
ward_area.drop(["ADM1_PCODE","ADM2_PCODE","ADM3_PCODE"],1,inplace=True)
ward_area.to_csv(join(proc_data_path,"ward_area.csv"),index=False)


In [ ]:
ward_4["latitude"]=ward_4.geometry.centroid.apply(lambda x: x.xy[1][0])
ward_4["longitude"]=ward_4.geometry.centroid.apply(lambda x: x.xy[0][0])
ward_4["ward_4_points"]=ward_4[["latitude","longitude"]].apply(lambda x: (x["latitude"],x["longitude"]),1).tolist()

ward_3["latitude"]=ward_3.geometry.centroid.apply(lambda x: x.xy[1][0])
ward_3["longitude"]=ward_3.geometry.centroid.apply(lambda x: x.xy[0][0])
ward_3["ward_3_points"]=ward_3[["latitude","longitude"]].apply(lambda x: (x["latitude"],x["longitude"]),1).tolist()

ward_2["latitude"]=ward_2.geometry.centroid.apply(lambda x: x.xy[1][0])
ward_2["longitude"]=ward_2.geometry.centroid.apply(lambda x: x.xy[0][0])
ward_2["ward_2_points"]=ward_2[["latitude","longitude"]].apply(lambda x: (x["latitude"],x["longitude"]),1).tolist()

ward_1["latitude"]=ward_1.geometry.centroid.apply(lambda x: x.xy[1][0])
ward_1["longitude"]=ward_1.geometry.centroid.apply(lambda x: x.xy[0][0])
ward_1["ward_1_points"]=ward_1[["latitude","longitude"]].apply(lambda x: (x["latitude"],x["longitude"]),1).tolist()


In [ ]:
# word_4_points=word_4[["latitude","longitude"]].apply(lambda x: (x["latitude"],x["longitude"]),1).tolist()
# word_4_cross_distance=cdist(word_4_points,word_4_points,metric=distance_m)
# save_pickle(word_4_cross_distance,join(other_data_path,"word_4_cross_distance.p"))
# word_4_cross_distance=read_pickle(join(other_data_path,"word_4_cross_distance.p"))
# len(word_4_points)

In [ ]:
# ward_3_points={ k:v for k,v in zip( ward_3.ADM3_PCODE ,ward_3.ward_3_points) }
ward_4["distance_to_ADM3"]=None
for k , v  in zip( ward_3.ADM3_PCODE ,ward_3.ward_3_points) : 
    ward_4.loc[ward_4.ADM3_PCODE==k,"distance_to_ADM3"]=cdist([v], ward_4.loc[ward_4.ADM3_PCODE==k,"ward_4_points"].tolist(),metric=distance_m)[0]


In [ ]:
ward_4["distance_to_ADM2"]=None
for k , v  in zip( ward_2.ADM2_PCODE ,ward_2.ward_2_points) : 
    ward_4.loc[ward_4.ADM2_PCODE==k,"distance_to_ADM2"]=cdist([v], ward_4.loc[ward_4.ADM2_PCODE==k,"ward_4_points"].tolist(),metric=distance_m)[0]

In [ ]:
ward_4["distance_to_ADM1"]=None
for k , v  in zip( ward_1.ADM1_PCODE ,ward_1.ward_1_points) : 
    ward_4.loc[ward_4.ADM1_PCODE==k,"distance_to_ADM1"]=cdist([v], ward_4.loc[ward_4.ADM1_PCODE==k,"ward_4_points"].tolist(),metric=distance_m)[0]

In [ ]:
word_4[["ADM4_PCODE","distance_to_ADM1","distance_to_ADM2","distance_to_ADM3"]].to_csv(join(proc_data_path,"ADM_distance.csv"),index=False)